# Data

In [ ]:
# R PACKAGES
if(!require("pacman")) install.packages("pacman")
pacman::p_load(arrow, tidyverse, janitor, readxl, stringdist)

In [ ]:
who_2019 <- read_delim(
  "icd102019syst_codes.txt", 
  delim = ";", 
  col_names=F, 
  show_col_types=F
) |> 
  select(
    who_code = X8, 
    who_desc = X9
  )

#CMS ICD10 CM code list
cms <- read_lines(
  "icd10cm_order_2019.txt") |> 
  as_tibble() |> 
  separate_wider_regex(
    value,
    patterns = c(
    id = "[^\\s]+",         # First non-whitespace block
    "\\s+",                 # One or more spaces
    rest = ".*"             # Everything else
      )
  ) |> 
  separate_wider_regex(
    rest,
    patterns = c(
      cms_code = "[^\\s]+",         # First non-whitespace block
      "\\s+",                 # One or more spaces
      rest_again = ".*"             # Everything else
    )
  ) |> 
  separate_wider_regex(
    rest_again,
    patterns = c(
      level = "[^\\s]+",         # First non-whitespace block
      "\\s+",                 # One or more spaces
      rest_desc = ".*"             # Everything else
    )
  ) |> 
  mutate(
    rest_desc = str_replace_all(rest_desc, "\\s{2,}", "!!!")
  ) |> 
    separate_wider_delim(
      rest_desc,
      delim = "!!!",
      too_few = "align_start",
      too_many = 'merge',
      names = c("cms_desc", "cms_desc_long")
    ) |> 
  select(cms_code, cms_desc) |> 
  distinct(cms_code, .keep_all = T)

In [ ]:
# HWA (thompson) codes
# getting the codes from the text extracted from the pdf to a tibble is  long and is shown in the hwa.r file
source("hwa.R")
hwa<- hwa_f() |> 
  mutate(
    code_length = str_count(hwa_code),
    code_letter = str_sub(hwa_code,1,1)
  ) |> 
  distinct(hwa_code, .keep_all = T) 
  # see whic codes have children/parents
  hwa <- hwa |> 
    mutate(
      has_child = map_lgl(hwa_code, function(code) {
      any(str_starts(hwa$hwa_code, code) & nchar(hwa$hwa_code) > nchar(code))
    }) ,
    is_child = map_lgl(hwa_code, function(code) {
      # Generate all prefixes of the code excluding the full code itself
      prefixes <- str_sub(code, 1, seq_len(str_length(code) - 1))
      # Check if any prefix exists in the list of codes
      any(prefixes %in% hwa$hwa_code)
    })
  ) |> 
    # I dont need the Z codes, these are not diagnoses
  filter(
    code_letter != "Z"
  )

write_csv (hwa, "hwa_codes.csv")

# Functions

In [4]:
describe_function <- function(df, col_name){

  print("N codes in list")
  df |> 
    count() |> 
    print()

  cat("\nCode lengths in list")
  df |> 
    group_by(code_length) |> 
    count() |> 
    print()

  cat("\nCode letters in list")
  df |> 
    group_by(code_letter) |>
    count() |>
    mutate(
      `%` = round(n/nrow(df)*100,1)
    ) |>
    print(n=120)

  cat("\nCode lengths and letters in list")
  df |> 
    group_by(code_length, code_letter) |>
    count() |>
    mutate(
      `%` = round(n/nrow(df)*100,1)
    ) |>
    ungroup() |> 
    arrange(code_letter, code_length) |> 
    print(n=120)
}

# Describe HWA code list

In [ ]:
describe_function(hwa, hwa_code)

## Parent and child codes

#### Describe the parent codes

In [ ]:
describe_function(hwa |> 
  filter(has_child), hwa_code)

In [ ]:
hwa |> 
  filter(has_child) |>
  print(n=120)

see which codes are children

In [ ]:
hwa |> 
  filter(is_child) |>
  print(n=120)

See which codes have 1-2-1 mapping in the CMS code list

In [9]:
hwa <- hwa |> 
  mutate(
    has_cms = map_lgl(hwa_code,  ~ .x %in% cms$cms_code)
  )

In [ ]:
print("N codes with 1-2-1 mapping")
hwa |>
  filter(has_cms) |>
  count() |>
  mutate(
    `%` = n/nrow(hwa)*100
  ) |>
  print()

# Mapping

## Parent Codes

I will only need to map the parent codes and the single codes.  
First, map the parent codes

In [ ]:
print("parents with 1-2-1 mapping")
hwa |>
  filter(has_cms) |>
  filter(has_child) |>
  count() |>
  mutate(
    `%` = n/nrow(hwa |> filter(has_child))*100
  ) |>
  print()


Most parent codes have 1-2-1 maps. Map these and check the mapping

In [ ]:
print("which parents have 1-2-1 mapping?")
parent_cms <- hwa |>
  filter(has_cms) |>
  filter(has_child) |>
  crossing(cms) |>
  filter(
    hwa_code == cms_code
  )
options(width=200)
parent_cms |> 
  select(hwa_code, hwa_desc, cms_desc, cms_code) |>
  print(n=120)

In [ ]:
describe_function(parent_cms, cms_code)

These look fine, the mappings look correct, and the sub-codes for these can be extracted becuase they are all going to be MSK (accoring to the HWA definition)

In [14]:
parent_cms_map <- parent_cms |>
  select(-cms_desc, -cms_code) |> 
  crossing(cms) |> 
  filter(
    startsWith(cms_code, hwa_code)
  )  

In [ ]:
describe_function(parent_cms_map, cms_code)

Now check out the parent codes that do not have 1-2-1 mapping

In [ ]:
hwa |>
  filter(!has_cms,has_child) |>
  print(n=120)

I use the [UMLS](https://uts.nlm.nih.gov/uts/umls/home) browser to find realted terms for these codes using the code description   
For M139 Arthritis unspecified the realted term is "Arthritis", which the UMLS maps down to ICD-10-CM M19.90 Arthritis Not Otherwise Specified.   
for M665 Spontaneous rupture unspecified tendon, the realted term is Non-traumatic tendon rupture, which UMLS maps to ICD-10-CM M66.9 Spontaneous rupture of unspecified tendon.   
So for these two codes, they can be mapped to the related terms

In [17]:
parent_cms_umls <- hwa |> 
  filter(!has_cms, has_child) |>
  mutate(
    code_to_map = case_when(
      hwa_code == "M139" ~ "M1990",
      hwa_code == "M665" ~ "M669"
    )
  ) |>
  crossing(cms) |>
  filter(
    startsWith(cms_code, code_to_map)
  ) |> 
  select(-code_to_map) 

parent_cms_join <- bind_rows(parent_cms_map, parent_cms_umls)

The child codes will not need to be mapped, as they've been mapped via their parent code

### Describe the parent/child code list

In [ ]:
describe_function(parent_cms_join, cms_code)

## Single codes

### Describe the single codes

In [ ]:
single_codes <- hwa |> 
  filter(!has_child & !is_child) 

describe_function(single_codes, hwa_code)

Now map the single codes and keep those with 1-2-1 mapping. Print these and manually check the maps

In [ ]:
single_cms <- single_codes |> 
  filter(has_cms) |> 
  crossing(cms) |>
  filter(
    hwa_code == cms_code
  ) 
options(width=200)
single_cms |> 
  select(hwa_code, hwa_desc, cms_desc, cms_code) |>
  print(n=400)

Describe the single codes with 1-2-1 maps

In [ ]:
describe_function(single_cms, hwa_code)

All of the single codes with 1-2-1 maps look OK, except T07. In ICD-10-CM, T07 has sub codes for Suicide attempt, and thus should not be included. This code will be removed. All other can be safely mapped, and thier sub-codes can be safely extracted. Map these and describe the dataset

In [ ]:
single_cms_mapped <- single_cms |> 
  filter(hwa_code != "T07") |> 
  select(-cms_desc, -cms_code) |>
  crossing(cms) |>
  filter(
    startsWith(cms_code, hwa_code)
  ) |> 
  # I dont need the codes I have already mapped using the parent codes
  anti_join(
    parent_cms_join,
    join_by(cms_code)
  )  
  
describe_function(single_cms_mapped, cms_code)

In [23]:
cms_join <- bind_rows(parent_cms_join, single_cms_mapped)

In [ ]:
describe_function(cms_join, cms_code)

Now examine the single codes without 1-2-1 mapping

In [ ]:
single_nocms <- hwa |> 
  filter(!has_cms, !has_child & !is_child)

describe_function(single_nocms, hwa_code)

In [ ]:
single_nocms |>
  print(n=120)

It is posisble that these codes could be mapped using their chapter, i.e. by taking the code 1 level up and seeing if   
- Step 1) those codes are suitable (MSK)    
and    
- Step 2) whether they have 1-2-1 CMS maps   

Step 1:

In [ ]:
single_nocms <- single_nocms |> 
  mutate(
    short_code = case_when(
      str_count(hwa_code) > 3 ~ str_sub(hwa_code, 1, -2),
      T ~ hwa_code
    )
  ) |> 
  left_join(
    who_2019,
    join_by(short_code == who_code)
  ) 

options(width=200)
single_nocms |>
  select(hwa_code,short_code, hwa_desc, who_desc) |>
  print(n=120)

THey all look fine. See how many have 1-2-1 mapping in the CMS list

In [ ]:
single_nocms_map <- single_nocms |> 
  select(hwa_code, hwa_desc, short_code, code_length, code_letter) |>
  crossing(cms) |>
  filter(
   cms_code == short_code
  )
options(width=200)
single_nocms_map |>
  select(hwa_code, hwa_desc, cms_desc, short_code,cms_code) |>
  print(n=120)

Of these, the following codes will need to be removed:
Z094 - not an MSK FU   
T141, T143,T144, T146: the mapping is too vague in ICD-10-CM, T14 contains subcodes for suicide   

In [ ]:
single_nocms_map <- single_nocms_map |>
  filter(
    !short_code %in% c("Z094", "T141", "T143", "T144", "T146")
  ) |> 
  select(hwa_code, hwa_desc, short_code, code_length, code_letter) |> 
  crossing(cms) |>
  filter(
    startsWith(cms_code, short_code)
  ) |> 
  select(-short_code) |> 
  # I dont need the codes I have already mapped using the parent  and single codes with 1-2-1 maps
  anti_join(
    cms_join,
    join_by(cms_code)
  ) 

describe_function(single_nocms_map, cms_code)
  

Create the final dataset. Use the allow combinations tribble to keep codes of the correct length for each code letter

In [30]:
cms_final <- bind_rows(cms_join, single_nocms_map) 

# Describe the final codes

In [ ]:
describe_function(cms_final, cms_code)

In [39]:
hwa_figure_a <- hwa |> 
  mutate(
    letter = str_sub(hwa_code, 1, 1)
  )  |> 
  group_by(letter) |>
  mutate(
    letter = factor(letter, 
    # levels = c("Not Mapped","M","S","T","D", "G","I","Q","R")
    ),
    n = round(n()/nrow(hwa)*100,1)
  ) |>
  ungroup() |> 
  distinct(letter, n) |> 
  mutate(
    code = "hwa"
  )

hwa_figure_b <- hwa |>
  left_join(
    cms_final,
      join_by(hwa_code)
  ) |> 
  filter(
    !is.na(cms_code)
  ) |>
  mutate(
    letter = ifelse(is.na(cms_code), "Not Mapped", str_sub(cms_code, 1, 1))
  ) |>
  group_by(letter) |>
  mutate(
    n = round(n()/nrow(cms_final)*100,1)
  ) |>
  ungroup() |>
  distinct(letter, n) |>
  mutate(
    letter = factor(
      letter, 
      # levels = c("Not Mapped","M","S","T","D", "G","I","Q","R")
    ),
    code = "CMS"
  )

hwa_figure <- hwa_figure_a |>
  bind_rows(hwa_figure_b) |> 
  mutate(
    code = ifelse(code == "hwa", "ICD-10-AM", "ICD-10-CM"),
    code = factor(
      code, 
      levels = c("ICD-10-AM", "ICD-10-CM")
    )
  )

# Save

In [40]:
write.csv(cms_join, "hwa_joined.csv")
write.csv(hwa_figure, "hwa_figure.csv")